In [432]:
import json
import os
import pandas as pd

In [433]:
# Define the directory path for saving CSV files
output_dir = 'src/data/csv/'

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [434]:
# Load the data from the JSON files
with open('src/data/json/characters.json', 'r', encoding='utf-8') as file:
    characters_data = json.load(file)[0]

with open('src/data/json/chapters.json', 'r', encoding='utf-8') as file:
    chapters_data = json.load(file)

In [435]:
characters = []

# Extract the data for each character
for character_key, character_value in characters_data.get('humans', {}).items():
    character = {'id': character_key, **character_value}
    characters.append(character)

# Convert the data to a DataFrame
characters_df = pd.DataFrame(characters)

# Save the data to a CSV file
characters_df.to_csv(output_dir + 'characters_test.csv', index=False)
characters_df.head()

,id,name,alias(es),gender,house(s),allegiance(s),title(s),family,race,culture(s),religion,physical description,birth,death,first appearance,last appearance
0,Aegon I Targaryen,Aegon Targaryen,"[{'book': 'A Game of Thrones', 'chapter': 3, '...",Male,"[{'book': 'A Game of Thrones', 'chapter': 0, '...",None,"[{'book': 'A Game of Thrones', 'chapter': 0, '...",{'father': 'Aerion Targaryen (son of Daemion)'...,Valyrian,"[{'book': 'A Game of Thrones', 'chapter': 0, '...",Faith of the Seven,"{'eye color': 'Purple', 'hair color': 'Silver-...","{'continent': 'Westeros', 'region': 'Crownland...","{'continent': 'Westeros', 'region': 'The Crown...",None,None
1,Aegon Targaryen (son of Rhaegar),Aegon Targaryen,"[{'book': 'A Clash of Kings', 'chapter': 48, '...",Male,"[{'book': 'A Game of Thrones', 'chapter': 0, '...","[{'book': 'A Game of Thrones', 'chapter': 0, '...","[{'book': 'A Game of Thrones', 'chapter': 0, '...","{'father': 'Rhaegar Targaryen', 'mother': 'Eli...",Valyrian,"[{'book': 'A Game of Thrones', 'chapter': 0, '...",None,"{'eye color': 'Purple', 'hair color': 'Silver-...","{'continent': 'Westeros', 'region': 'Crownland...","{'continent': 'Westeros', 'region': 'The Crown...",None,None
2,Aerys II Targaryen,Aerys Targaryen,"[{'book': 'A Game of Thrones', 'chapter': 2, '...",Male,"[{'book': 'A Game of Thrones', 'chapter': 0, '...",None,"[{'book': 'A Game of Thrones', 'chapter': 0, '...","{'father': 'Jaehaerys II Targaryen', 'mother':...",Valyrian,"[{'book': 'A Game of Thrones', 'chapter': 0, '...",Faith of the Seven,"{'eye color': 'Purple', 'hair color': 'Silver-...","{'continent': 'Westeros', 'region': 'Crownland...","{'continent': 'Westeros', 'region': 'The Crown...",None,None
3,Arya Stark,Arya Stark,"[{'book': 'A Game of Thrones', 'chapter': 7, '...",Female,"[{'book': 'A Game of Thrones', 'chapter': 0, '...","[{'book': 'A Game of Thrones', 'chapter': 0, '...","[{'book': None, 'chapter': None, 'title': 'Pri...","{'father': 'Eddard Stark', 'mother': 'Catelyn ...",First Men,"[{'book': 'A Game of Thrones', 'chapter': 0, '...",Old Gods of the Forest,"{'eye color': 'Grey', 'hair color': 'Brown'}","{'continent': 'Westeros', 'region': 'The North...",None,"{'book': 'A Game of Thrones', 'chapter': 0}","{'book': 'A Game of Thrones', 'chapter': 10}"
4,Ayrmidon,Ayrmidon,None,Male,None,None,None,"{'father': None, 'mother': None, 'sibling(s)':...",None,"[{'book': 'A Game of Thrones', 'chapter': 0, '...",None,"{'eye color': None, 'hair color': None}",None,None,None,None


In [436]:
def flatten_or_none(nested_lists):
    flat = []

    def recurse(items):
        if items:
            for item in items:
                if isinstance(item, list):
                    recurse(item)
                elif item is not None:
                    flat.append(item)
    
    recurse(nested_lists)

    return flat if flat else None

def extract_values(item, key):
    """
    Extracts values associated with a specific key from a list of dictionaries or a single dictionary.
    
    Parameters:
    - item: The input data, which can be a list of dictionaries, a single dictionary, or another type.
    - key: The key whose associated values need to be extracted.
    
    Returns:
    - A list of extracted values if the input is a list of dictionaries.
    - A single value if the input is a dictionary.
    - The original item if it's neither a list nor a dictionary.
    """
    if not item:
        return None

    if isinstance(key, tuple):
        if isinstance(item, list):
            return flatten_or_none([extract_values(element.get(key[0]), key[1]) if isinstance(element, dict) else element for element in item])
    
        if isinstance(item, dict):
            return extract_values(item.get(key[0]), key[1])
    
    if isinstance(key, str):
        if isinstance(item, list):
            return flatten_or_none([element.get(key) if isinstance(element, dict) else element for element in item])
    
        if isinstance(item, dict):
            return item.get(key)
    
    if isinstance(key, list):
        results = []
        for k in key:
            results.append(extract_values(item, k))

        return flatten_or_none(results)

def replace_columns(df, old_col, new_cols):
    """
    Replaces an existing column with multiple new columns while preserving the column order.
    
    Parameters:
    - df: The pandas DataFrame.
    - old_col: The name of the column to be replaced.
    - new_cols: A dictionary where keys are new column names and values are the data for these columns.
    
    Returns:
    - None. The DataFrame is modified in place.
    """
    if old_col not in df.columns:
        return
    # Get the index of the column to be replaced
    col_idx = df.columns.get_loc(old_col)
    # Drop the old column
    df.drop(columns=[old_col], inplace=True)
    # Insert each new column at the correct position
    for i, (new_col, data) in enumerate(new_cols.items()):
        df.insert(col_idx + i, new_col, data)

In [437]:
# Mapping of columns to transform and their corresponding keys to extract
columns_to_extract = {
    'title(s)': 'title',
    'house(s)': 'house',
    'culture(s)': 'culture',
    'allegiance(s)': 'allegiance',
    'alias(es)': 'alias',
}

# Mapping for 'family' related columns
family_columns = {
    'father': 'father',
    'mother': 'mother',
    'sibling(s)': ('sibling(s)', 'sibling'),
    'child(ren)': ('child(ren)', 'child'),
    'spouse(s)': ('spouse(s)', 'spouse'),
    'lover(s)': ('lover(s)', 'lover'),
}

# Mapping for location-related columns
location_columns = {
    'birth': {
        'birth_continent': 'continent',
        'birth_region': 'region',
        'birth_location': 'location',
        'birth_date': 'date'
    },
    'death': {
        'death_continent': 'continent',
        'death_region': 'region',
        'death_location': 'location',
        'death_date': 'date',
        'death_reason': ('by', 'type'),
        'killed_by': ('by', 'name')
    }
}

# Mapping for appearance-related columns
appearance_columns = {
    'first appearance': {
        'first_appearance_book': 'book',
        'first_appearance_chapter': 'chapter'
    },
    'last appearance': {
        'last_appearance_book': 'book',
        'last_appearance_chapter': 'chapter'
    }
}

# Mapping for physical description columns
physical_description_columns = {
    'eye color': 'eye color',
    'hair color': 'hair color'
}

# Transform simple columns based on the mappings
for col, key in columns_to_extract.items():
    if col in characters_df.columns:
        characters_df[col] = characters_df[col].apply(lambda x: extract_values(x, key))

# Transform 'family' related columns
if 'family' in characters_df.columns:
    new_family_cols = {}
    for new_col, key in family_columns.items():
        # Extract the value using the specified key
        new_family_cols[new_col] = characters_df['family'].apply(lambda family: extract_values(family, key) if isinstance(family, dict) else None)
    replace_columns(characters_df, 'family', new_family_cols)

# Transform physical description columns
if 'physical description' in characters_df.columns:
    new_physical_cols = {}
    for new_col, key in physical_description_columns.items():
        new_physical_cols[new_col] = characters_df['physical description'].apply(lambda item: extract_values(item, key))
    replace_columns(characters_df, 'physical description', new_physical_cols)

# Transform location-related columns
for old_col, mappings in location_columns.items():
    if old_col in characters_df.columns:
        new_cols = {}
        for new_col, key in mappings.items():
            new_cols[new_col] = characters_df[old_col].apply(lambda item: extract_values(item, key))
        replace_columns(characters_df, old_col, new_cols)

# Transform appearance-related columns
for old_col, mappings in appearance_columns.items():
    if old_col in characters_df.columns:
        new_cols = {}
        for new_col, key in mappings.items():
            new_cols[new_col] = characters_df[old_col].apply(lambda item: extract_values(item, key))
        replace_columns(characters_df, old_col, new_cols)

# Save the transformed data to a CSV file
characters_df.to_csv(output_dir + 'characters_test.csv', index=False)
characters_df.head()

,id,name,alias(es),gender,house(s),allegiance(s),title(s),father,mother,sibling(s),...,death_continent,death_region,death_location,death_date,death_reason,killed_by,first_appearance_book,first_appearance_chapter,last_appearance_book,last_appearance_chapter
0,Aegon I Targaryen,Aegon Targaryen,"[Aegon the Conqueror, Aegon the Dragonlord, Ae...",Male,[Targaryen],None,"[Lord of Dragonstone, King of All Westeros, Sh...",Aerion Targaryen (son of Daemion),Valaena Velaryon,"[Rhaenys Targaryen, Visenya Targaryen]",...,Westeros,The Crownlands,Dragonstone,37 AC,Nature,Stroke,None,NaN,None,NaN
1,Aegon Targaryen (son of Rhaegar),Aegon Targaryen,[The prince that was promised],Male,[Targaryen],[House Targaryen],[Prince],Rhaegar Targaryen,Elia Martell,[Rhaenys Targaryen (daughter of Rhaegar)],...,Westeros,The Crownlands,King's Landing,283 AC,Character,Gregor Clegane,None,NaN,None,NaN
2,Aerys II Targaryen,Aerys Targaryen,"[The Mad King, Mad King Aerys, King Scab, Aery...",Male,[Targaryen],None,"[Prince of Dragonstone, King of the Andals, th...",Jaehaerys II Targaryen,Shaera Targaryen,[Rhaella Targaryen],...,Westeros,The Crownlands,King's Landing,283 AC,Character,Jaime Lannister,None,NaN,None,NaN
3,Arya Stark,Arya Stark,"[Arya Horseface, Arya Underfoot, Arry, Lumpyhe...",Female,[Stark],"[House Stark, Faceless Men]","[Princess, Lady of Winterfell, Acolyte]",Eddard Stark,Catelyn Stark,"[Bran Stark, Jon Snow, Rickon Stark, Robb Star...",...,None,None,None,None,None,None,A Game of Thrones,0.0,A Game of Thrones,10.0
4,Ayrmidon,Ayrmidon,None,Male,None,None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,None,NaN


In [438]:
chapters = []

# Extract the data for each chapter
for book in chapters_data:
    for chapter in book.get('chapters', {}):
        chapter = {'book': book.get('title'), 'book_author': book.get('author'), 'book_published': book.get('published'), 'book_pages': book.get('pages'), **chapter}
        chapters.append(chapter)

# Convert the data to a DataFrame
chapters_df = pd.DataFrame(chapters)

# Save the data to a CSV file
chapters_df.to_csv(output_dir + 'chapters_test.csv', index=False)
chapters_df.head()

,book,book_author,book_published,book_pages,title,number,pov,pages,words,year,theme(s),characters,continents,groups,events
0,None,George R. R. Martin,1996-08-06,None,Prologue,0,Will,11,None,297,None,"{'direwolves': None, 'humans': {'appearing': [...","{'appearing': [{'name': 'Westeros', 'regions':...","{'factions': None, 'houses': {'appearing': ['H...",None
1,None,George R. R. Martin,1996-08-06,None,Bran I,1,Bran Stark,9,None,298,None,"{'direwolves': {'appearing': ['Ghost', 'Grey W...","{'appearing': [{'name': 'Westeros', 'regions':...","{'factions': None, 'houses': {'appearing': ['H...","{'appearing': None, 'mentioned': [{'type': 'Pe..."
2,None,George R. R. Martin,1996-08-06,None,Catelyn I,2,Catelyn Stark,6,None,298,None,"{'direwolves': None, 'humans': {'appearing': [...","{'appearing': [{'name': 'Westeros', 'regions':...","{'factions': None, 'houses': {'appearing': ['H...","{'appearing': None, 'mentioned': [{'type': 'Ca..."
3,None,George R. R. Martin,1996-08-06,None,Daenerys I,3,Daenerys Targaryen,11,None,298,None,"{'direwolves': None, 'humans': {'appearing': [...","{'appearing': [{'name': 'Essos', 'regions': {'...","{'factions': None, 'houses': {'appearing': ['H...","{'appearing': None, 'mentioned': [{'type': 'Ba..."
4,None,George R. R. Martin,1996-08-06,None,Eddard I,4,Eddard Stark,10,None,298,None,"{'direwolves': None, 'humans': {'appearing': [...","{'appearing': [{'name': 'Westeros', 'regions':...","{'factions': None, 'houses': {'appearing': ['H...","{'appearing': None, 'mentioned': [{'type': 'Wa..."


In [439]:
# Mapping for 'characters' related columns
character_columns = {
    'appearing_direwolves': ('direwolves', 'appearing'),
    'mentioned_direwolves': ('direwolves', 'mentioned'),
    'appearing_humans': ('humans', 'appearing'),
    'mentioned_humans': ('humans', 'mentioned'),
}

location_columns = {
    'appearing_continents': ('appearing', 'name'),
    'mentioned_continents': ('mentioned', 'name'),

    'appearing_regions': ('appearing', ('regions', ('appearing', 'name'))),
    'mentioned_regions': [
        ('mentioned', ('regions', ('mentioned', 'name'))),
        ('appearing', ('regions', ('mentioned', 'name')))
    ],

    'appearing_locations': ('appearing', ('regions', ('appearing', ('locations', ('appearing', 'name'))))),
    'mentioned_locations': [
        ('appearing', ('regions', ('appearing', ('locations', ('mentioned', 'name'))))),
        ('appearing', ('regions', ('mentioned', ('locations', ('mentioned', 'name'))))),
        ('mentioned', ('regions', ('mentioned', ('locations', ('mentioned', 'name'))))),
    ],
}

group_columns = {
    'appearing_factions': ('factions', 'appearing'),
    'mentioned_factions': ('factions', 'mentioned'),

    'appearing_houses': ('houses', 'appearing'),
    'mentioned_houses': ('houses', 'mentioned'),

    'appearing_mercenary_companies': ('mercenary companies', 'appearing'),
    'mentioned_mercenary_companies': ('mercenary companies', 'mentioned'),

    'appearing_military_orders': ('military orders', 'appearing'),
    'mentioned_military_orders': ('military orders', 'mentioned'),

    'appearing_other_orders': ('other orders', 'appearing'),
    'mentioned_other_orders': ('other orders', 'mentioned'),

    'appearing_peoples': ('peoples', 'appearing'),
    'mentioned_peoples': ('peoples', 'mentioned'),

    'appearing_religious_orders': ('religious orders', 'appearing'),
    'mentioned_religious_orders': ('religious orders', 'mentioned'),
}

event_columns = {
    'appearing_events_types': ('appearing', 'type'),
    'appearing_events_names': ('appearing', 'name'),
    'mentioned_events_types': ('mentioned', 'type'),
    'mentioned_events_names': ('mentioned', 'name'),
}

# Transform 'characters' related columns
if 'characters' in chapters_df.columns:
    new_characters_cols = {}
    for new_col, key in character_columns.items():
        # Extract the value using the specified key
        new_characters_cols[new_col] = chapters_df['characters'].apply(lambda characters: extract_values(characters, key) if isinstance(characters, dict) else None)
    replace_columns(chapters_df, 'characters', new_characters_cols)

# Transform 'continents' related columns
if 'continents' in chapters_df.columns:
    new_contients_cols = {}
    for new_col, key in location_columns.items():
        # Extract the value using the specified key
        new_contients_cols[new_col] = chapters_df['continents'].apply(lambda contients: extract_values(contients, key) if isinstance(contients, dict) else None)
    replace_columns(chapters_df, 'continents', new_contients_cols)

# Transform 'groups' related columns
if 'groups' in chapters_df.columns:
    new_groups_cols = {}
    for new_col, key in group_columns.items():
        # Extract the value using the specified key
        new_groups_cols[new_col] = chapters_df['groups'].apply(lambda groups: extract_values(groups, key) if isinstance(groups, dict) else None)
    replace_columns(chapters_df, 'groups', new_groups_cols)

# Transform 'events' related columns
if 'events' in chapters_df.columns:
    new_events_cols = {}
    for new_col, key in event_columns.items():
        # Extract the value using the specified key
        new_events_cols[new_col] = chapters_df['events'].apply(lambda events: extract_values(events, key) if isinstance(events, dict) else None)
    replace_columns(chapters_df, 'events', new_events_cols)

# Save the transformed data to a CSV file
chapters_df.to_csv(output_dir + 'chapters_test.csv', index=False)
chapters_df.head(11)

,book,book_author,book_published,book_pages,title,number,pov,pages,words,year,...,appearing_other_orders,mentioned_other_orders,appearing_peoples,mentioned_peoples,appearing_religious_orders,mentioned_religious_orders,appearing_events_types,appearing_events_names,mentioned_events_types,mentioned_events_names
0,None,George R. R. Martin,1996-08-06,None,Prologue,0,Will,11,None,297,...,None,[Maesters],[Others],[Wildlings],None,None,None,None,None,None
1,None,George R. R. Martin,1996-08-06,None,Bran I,1,Bran Stark,9,None,298,...,None,None,None,"[Others, Wildlings]",None,None,None,None,[Period],[Long Night]
2,None,George R. R. Martin,1996-08-06,None,Catelyn I,2,Catelyn Stark,6,None,298,...,None,[Maesters],None,"[Children of the Forest, Others, Wildlings]",None,[Septons],None,None,[Cataclysm],[Doom of Valyria]
3,None,George R. R. Martin,1996-08-06,None,Daenerys I,3,Daenerys Targaryen,11,None,298,...,None,None,[Dothraki],None,None,[Red Priests],None,None,"[Battle, Cataclysm]","[Battle of the Trident, Doom of Valyria]"
4,None,George R. R. Martin,1996-08-06,None,Eddard I,4,Eddard Stark,10,None,298,...,None,None,None,[Others],None,None,None,None,[War],[Greyjoy's Rebellion]
5,None,George R. R. Martin,1996-08-06,None,Jon I,5,Jon Snow,9,None,298,...,None,None,None,None,None,None,None,None,[War],[Conquest of Dorne]
6,None,George R. R. Martin,1996-08-06,None,Catelyn II,6,Catelyn Stark,10,None,298,...,[Maesters],None,None,[Others],None,None,None,None,None,None
7,None,George R. R. Martin,1996-08-06,None,Arya I,7,Arya,8,None,298,...,None,None,None,None,None,[Septas],None,None,None,None
8,None,George R. R. Martin,1996-08-06,None,Bran II,8,Bran Stark,10,None,298,...,None,[Maesters],None,None,None,None,None,None,[War],[Dance of the Dragons]
9,None,George R. R. Martin,1996-08-06,None,Tyrion I,9,Tyrion Lannister,7,None,298,...,None,[Maesters],None,None,[Septons],None,None,None,None,None
